In [130]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pickle

In [131]:
all_songs = pd.read_csv('all_songs.csv')
some_songs = all_songs.loc[all_songs['lyrics'].str.startswith('[').fillna(False)]
some_songs

,song title,lyrics,artist
0,In Da Club,"[Intro]\nGo, go, go, go, go, go\nGo Shorty, it...",50 Cent
1,21 Questions,[Ad-Libs]\nNew York City\nYou are now rockin'\...,50 Cent
2,Many Men (Wish Death),"[Skit]\nMan, we gotta go get somethin' to eat\...",50 Cent
3,My Life,"[Chorus]\nMy, yeah, yeah, mmm\nMy life, my lif...",50 Cent
4,Patiently Waiting,"[Intro]\nHey Em, you know you're my favorite w...",50 Cent
...,...,...,...
575,Cruisin’,"[Intro]\n(Crusin')\n\n[Verse 1]\nBaby, let's c...",Smokey Robinson
576,Really Gonna Miss You,[Verse 1]\nReally gonna miss you\nIt's really ...,Smokey Robinson
579,The Agony and the Ecstasy,[Verse 1]\nWhat's it all about this crazy love...,Smokey Robinson
582,Ooh Baby Baby,[Verse 1]\nI did you wrong\nMy heart went out ...,Smokey Robinson


In [132]:
def split_by_verse(song):
    verses = song.split('[')
    for ind in range(1, len(verses)):
        verses[ind] = verses[ind].split(']')[1]
        verses[ind] = verses[ind][1:]
        verses[ind] = re.sub('\n', ' ', verses[ind])
        verses[ind] = re.sub('  ', '', verses[ind])
    return verses[1:]

def get_embeddings(msgs):
    encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
    embeddings = encoder.encode(msgs)
    return torch.unsqueeze(torch.from_numpy(embeddings), 0)

In [133]:
X = some_songs.lyrics.values
y = pd.get_dummies(some_songs.artist.values).to_numpy()

In [134]:
X = X[:100:5]
y = y[:100:5]

for ind in tqdm(range(len(X))):
    try:
        verses = split_by_verse(X[ind])
        X[ind] = get_embeddings(verses)
    except IndexError:
        # remove the y entry for failed X
        np.delete(y, ind, 0)
        continue

100%|██████████| 20/20 [00:36<00:00,  1.85s/it]


In [135]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, shuffle = True)

In [136]:
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

torch.Size([18, 13])

In [173]:
class BaseLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(BaseLSTM, self).__init__()
        
        # define hidden size
        self.hidden_size = hidden_size

        # define lstm layer
        self.lstm = nn.LSTM(input_size, hidden_size, 1, batch_first = True)

        # define fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        
        # softmax activation
        self.softmax = nn.Softmax(dim=0)

    def forward(self, input):
        
        #init hidden states
        h0 = torch.zeros(1, 1, self.hidden_size)
        c0 = torch.zeros(1, 1, self.hidden_size)
        
        # pass input and h0, c0 into LSTM
        out, (h_out, c_out) = self.lstm(input, (h0, c0))
        
        out = out.reshape(-1, self.hidden_size)
        out = self.fc(out)
        artist_scores = self.softmax(out[-1])
        artist_scores = torch.unsqueeze(artist_scores, 0)
        return artist_scores

In [175]:
# training
model = BaseLSTM(input_size=768, hidden_size=256, output_size=13)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

model.train()
for e in range(300):
    for x, y in zip(x_train, y_train):
        model.zero_grad()

        artist_scores = model(x)

        loss = loss_function(artist_scores, torch.unsqueeze(torch.argmax(y), 0))
        loss.backward()
        optimizer.step()
    if (e%50) == 0:
        print('Running epoch... {}'.format(e))

Running epoch 0
Running epoch 50
Running epoch 100
Running epoch 150
Running epoch 200
Running epoch 250


In [190]:
def Accuracy(xs, ys):
    correct = 0
    for i in range(len(xs)):
        # run through model
        pred = model(xs[i])
        # softmax distribution
        pred = nn.functional.softmax(pred[-1], dim=0).data
        # calc argmax
        pred = torch.argmax(pred).item()
        # sum up correct predictions
        correct += (pred == torch.unsqueeze(torch.argmax(ys[i]), 0))
    return correct.item()/len(xs)

test_acc = Accuracy(x_test, y_test)
print('{}% Test Accuracy'.format(test_acc*100))

50.0% Test Accuracy


In [191]:
train_acc = Accuracy(x_train, y_train)
print('{}% Train Accuracy'.format(train_acc*100))

50.0% Train Accuracy
